<a href="https://colab.research.google.com/github/salim4n/kaggle_competition/blob/main/BibleGpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bible Gpt

## Nettoyage data

In [4]:
with open('/content/fra-fraLSG.txt') as f:
    lines = f.readlines()

In [5]:
corpus = []
for line in lines:
  line = line.split('\n')[0]
  line = line.lower()
  if line:
    corpus.append(line)

In [6]:
len(corpus)

31055

In [7]:
corpus = corpus[:7000]

## Tokenisation

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
tokenizer = Tokenizer()

In [10]:
tokenizer.fit_on_texts(corpus)

In [11]:
len(tokenizer.word_index)

9420

In [12]:
corpus[3]

'dieu vit que la lumière était bonne; et dieu sépara la lumière d’avec les ténèbres.'

In [13]:
seq = tokenizer.texts_to_sequences([corpus[3]])[0]

In [14]:
seq

[27, 279, 10, 4, 1840, 78, 1102, 1, 27, 2660, 4, 1840, 2284, 5, 1542]

In [15]:
for i in range(1, len(seq)):
  print(seq[:i+1])

[27, 279]
[27, 279, 10]
[27, 279, 10, 4]
[27, 279, 10, 4, 1840]
[27, 279, 10, 4, 1840, 78]
[27, 279, 10, 4, 1840, 78, 1102]
[27, 279, 10, 4, 1840, 78, 1102, 1]
[27, 279, 10, 4, 1840, 78, 1102, 1, 27]
[27, 279, 10, 4, 1840, 78, 1102, 1, 27, 2660]
[27, 279, 10, 4, 1840, 78, 1102, 1, 27, 2660, 4]
[27, 279, 10, 4, 1840, 78, 1102, 1, 27, 2660, 4, 1840]
[27, 279, 10, 4, 1840, 78, 1102, 1, 27, 2660, 4, 1840, 2284]
[27, 279, 10, 4, 1840, 78, 1102, 1, 27, 2660, 4, 1840, 2284, 5]
[27, 279, 10, 4, 1840, 78, 1102, 1, 27, 2660, 4, 1840, 2284, 5, 1542]


In [16]:
sequences = tokenizer.texts_to_sequences(corpus)

In [17]:
input_sequences = []
for seq in sequences:
  for i in range(1, len(seq)):
    data_line = seq[:i+1]
    input_sequences.append(data_line)

In [18]:
(input_sequences[7:12])

[[29, 2046, 27, 1541, 5, 549, 1, 4, 91],
 [4, 91],
 [4, 91, 78],
 [4, 91, 78, 5561],
 [4, 91, 78, 5561, 1]]

## Padding

In [19]:
max([len(x) for x in input_sequences])

66

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
input_sequences = pad_sequences(input_sequences, padding="pre", maxlen=66)

## Creation x_train et y_train

In [22]:
x_train = input_sequences[:, :-1]
y_train = input_sequences[:, -1]

In [23]:
x_train

array([[   0,    0,    0, ...,    0,    0,   29],
       [   0,    0,    0, ...,    0,   29, 2046],
       [   0,    0,    0, ...,   29, 2046,   27],
       ...,
       [   0,    0,    0, ..., 1372,   41,   24],
       [   0,    0,    0, ...,   41,   24,   10],
       [   0,    0,    0, ...,   24,   10,  475]], dtype=int32)

In [24]:
y_train

array([2046,   27, 1541, ...,   10,  475,   11], dtype=int32)

In [25]:
import tensorflow as tf

In [26]:
tf.keras.utils.to_categorical([4,2,1])

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [27]:
y_train = tf.keras.utils.to_categorical(y_train)

In [28]:
y_train.shape

(163770, 9421)

## Modeling

In [29]:
import numpy as np

In [30]:
np.power(66, 1/4)

2.8502698827717983

In [31]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Embedding(9421, 16),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(9421, activation='softmax'),

    ]
)

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          150736    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              41472     
 l)                                                              
                                                                 
 dense (Dense)               (None, 9421)              1215309   
                                                                 
Total params: 1,407,517
Trainable params: 1,407,517
Non-trainable params: 0
_________________________________________________________________


In [33]:
h = model.fit(x_train, y_train, epochs=20, batch_size=1024)

Epoch 1/20
160/160 [==============================] - 38s 175ms/step - loss: 6.9211 - accuracy: 0.0464
Epoch 2/20
160/160 [==============================] - 18s 114ms/step - loss: 6.4261 - accuracy: 0.0480
Epoch 3/20
160/160 [==============================] - 13s 80ms/step - loss: 6.2375 - accuracy: 0.0562
Epoch 4/20
160/160 [==============================] - 12s 74ms/step - loss: 6.0917 - accuracy: 0.0678
Epoch 5/20
160/160 [==============================] - 11s 68ms/step - loss: 5.9881 - accuracy: 0.0739
Epoch 6/20
160/160 [==============================] - 12s 74ms/step - loss: 5.9175 - accuracy: 0.0777
Epoch 7/20
160/160 [==============================] - 9s 59ms/step - loss: 5.8616 - accuracy: 0.0807
Epoch 8/20
160/160 [==============================] - 9s 58ms/step - loss: 5.8001 - accuracy: 0.0856
Epoch 9/20
160/160 [==============================] - 9s 58ms/step - loss: 5.7264 - accuracy: 0.0915
Epoch 10/20
160/160 [==============================] - 9s 58ms/step - loss: 5.6450 

## Generer du texte

In [34]:
prompt = "dieu est"

In [35]:
prompt_seq = tokenizer.texts_to_sequences([prompt])
prompt_seq

[[27, 45]]

In [36]:
prompt_seq[0]

[27, 45]

In [37]:
prompt_x = pad_sequences([prompt_seq[0]], padding="pre", maxlen=65)

In [38]:
prompt_x.shape

(1, 65)

In [39]:
prompt_x

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 27,
        45]], dtype=int32)

In [40]:
pred = model.predict(prompt_x)

1/1 [==============================] - 1s 1s/step


In [41]:
np.argmax(pred)

4

In [42]:
tokenizer.index_word

{1: 'et',
 2: 'de',
 3: 'le',
 4: 'la',
 5: 'les',
 6: 'à',
 7: 'il',
 8: 'l’éternel',
 9: 'des',
 10: 'que',
 11: 'vous',
 12: 'qui',
 13: 'tu',
 14: 'pour',
 15: 'en',
 16: 'du',
 17: 'dans',
 18: 'un',
 19: 'ne',
 20: 'je',
 21: 'ce',
 22: 'sur',
 23: 'fils',
 24: 'dit',
 25: 'ils',
 26: 'son',
 27: 'dieu',
 28: 'lui',
 29: 'au',
 30: 'avec',
 31: 'point',
 32: 'pays',
 33: 'une',
 34: 'ton',
 35: 'se',
 36: 'moïse',
 37: 'sa',
 38: 'nous',
 39: 'a',
 40: 'ses',
 41: 'leur',
 42: 'd’israël',
 43: 'enfants',
 44: 'tout',
 45: 'est',
 46: 'pas',
 47: 'tous',
 48: 'par',
 49: 'leurs',
 50: 'toi',
 51: 'sera',
 52: 'comme',
 53: 'devant',
 54: 'car',
 55: 'peuple',
 56: 'c’est',
 57: 'qu’il',
 58: 'elle',
 59: 'te',
 60: 'si',
 61: 'avait',
 62: 'moi',
 63: 'deux',
 64: 'tes',
 65: 'aux',
 66: 'jour',
 67: 'mon',
 68: 'y',
 69: 'mais',
 70: 'voici',
 71: 'fait',
 72: 'on',
 73: 'père',
 74: 'femme',
 75: 'sont',
 76: 'vos',
 77: 'fit',
 78: 'était',
 79: 'votre',
 80: 'ces',
 81: 'selon

In [43]:

tokenizer.index_word.get(18)

'un'

In [44]:
n_predict = 10

for _ in range(n_predict):
  prompt_seq = tokenizer.texts_to_sequences([prompt])
  prompt_x = pad_sequences([prompt_seq[0]], padding="pre", maxlen=65)
  pred = model.predict(prompt_x)
  index = np.argmax(pred)
  mot_predit = tokenizer.index_word.get(index)
  print(prompt + " " + mot_predit)
  prompt = prompt + " " + mot_predit


1/1 [==============================] - 0s 40ms/step
dieu est la
1/1 [==============================] - 0s 63ms/step
dieu est la maison
1/1 [==============================] - 0s 68ms/step
dieu est la maison de
1/1 [==============================] - 0s 22ms/step
dieu est la maison de l’éternel
1/1 [==============================] - 0s 22ms/step
dieu est la maison de l’éternel et
1/1 [==============================] - 0s 21ms/step
dieu est la maison de l’éternel et à
1/1 [==============================] - 0s 25ms/step
dieu est la maison de l’éternel et à l’éternel
1/1 [==============================] - 0s 21ms/step
dieu est la maison de l’éternel et à l’éternel et
1/1 [==============================] - 0s 25ms/step
dieu est la maison de l’éternel et à l’éternel et je
1/1 [==============================] - 0s 21ms/step
dieu est la maison de l’éternel et à l’éternel et je suis
